<a href="https://colab.research.google.com/github/rathpr3/xgboost/blob/main/llama_finetuning_myattempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-cr

In [4]:
!pip install wandb -qU
import wandb
wandb.login(key='')
#128f1b5a52be79cf88f12bb7f8ad385a348afbc2

wandb: Currently logged in as: prarathore17. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"

#dataset_name = 'AlexanderDoria/novel17_test' #french novels
dataset = load_dataset(dataset_name, split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

#model_name = "abhishek/llama-2-7b-hf-small-shards"
#model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# load model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map = "auto",
    use_cache=False,
)
#model.config.use_cache = False




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# load tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

In [8]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [10]:
from peft import LoraConfig, get_peft_model

lora_alpha = 32
lora_dropout = 0.05
lora_r = 8 #Increasing r (not more than 32) will lead to more robust model but higher memory consumption at the same time.

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],   #target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
print_trainable_parameters(model)



trainable params: 8388608 || all params: 3508801536 || trainable%: 0.23907331075678143


In [11]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [12]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [13]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [14]:
trainer.train()

wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230918_102334-zcs76cxf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run upbeat-galaxy-3
wandb: ⭐️ View project at https://wandb.ai/prarathore17/huggingface
wandb: 🚀 View run at https://wandb.ai/prarathore17/huggingface/runs/zcs76cxf
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.356300
20,1.219800
30,1.215200
40,1.426600
50,1.589500
60,1.182000
70,1.205200
80,1.227500
90,1.437800
100,1.580400


TrainOutput(global_step=100, training_loss=1.3440285968780517, metrics={'train_runtime': 609.7357, 'train_samples_per_second': 2.624, 'train_steps_per_second': 0.164, 'total_flos': 1.123324756328448e+16, 'train_loss': 1.3440285968780517, 'epoch': 0.16})

In [15]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [16]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [24]:
text = "How to take care of indoor plants?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs) #max_new_tokens=50
print(tokenizer.decode(outputs[0], skip_special_tokens=True,max_length=5000))

How to take care of indoor plants?
 everybody knows that indoor plants are a great way to add some greenery to your home, but did you know that they can also help purify the air, improve mood, and even reduce stress? Here are some tips for taking care of your indoor plants.
First, make sure you're choosing the right plants for your home. Some popular options include snake plants, spider plants, and peace lilies. These plants are easy to care for and can thrive in low-light conditions, which is perfect for those who don't have a lot of natural light in their homes.
Next, learn about the specific needs of your plants. Different plants have different watering needs, so make sure you're not over- or under-watering them. For example, cacti and succulents prefer to dry out between waterings, while ferns and mosses like to stay moist.
Also, pay attention to the soil. Make sure it's well-draining and not too dense, as this can cause roots to rot. You can also add organic matter like peat moss 